Day 7: Figuring out directory sizes from command line input/output (ugh)

In [152]:
import datetime

In [144]:
input = []

with open('day7-input.txt', 'r') as f:
    for line in f.readlines():
        input.append(line.rstrip())

input[0], input[-1], input[1], input[2]

('$ cd /', '43248 jwdv.qct', '$ ls', 'dir drblq')

In [128]:
def is_command(string):
    return string[0] == "$"

def is_calling_ls(string):
    return string[2:4] == "ls"

def is_a_dir(string):
    return string[0:3] == "dir"

def is_calling_cd(string):
    return string[2:4] == "cd"

def has_size(string):
    return string[0] in [*'0123456789']

is_calling_ls('$ ls'), is_command("$   a sd as d as"), is_a_dir('dir drblq'), is_calling_cd('$ cd /'), has_size('43251 jw.jpg'), has_size('$ asdas')

(True, True, True, True, True, False)

Plan:

-go through input,

-find an ls call:

------add up all the sizes from lines with size
    (ignoring dirs then)
    (UNTIL another command called)

In [ ]:
counting = False
totals = []
total = 0

for line in input:
    if counting:
        if has_size(line):
            tmp = line.split()
            total += int(tmp[0])
        elif is_a_dir(line):
            pass
        elif is_command(line):
            if total <= 100000 and total != 0:
                totals.append(total)
                total = 0
                counting = False
            else:
                total = 0
                counting = False
        else:
            print('panic')
    else:
        if is_calling_ls(line):
            counting = True

sum(totals)
        

THat didn't work

Need to track each individual directory and store answers in there

so: GO through, have to know WHERE we are.

if an ls: where was that ls called (what dir!)

In [157]:
counting = False
totals = {}

total = 0
where = ''
all_dirs = []

for line in input:
    if is_calling_cd(line):  # calling cd -> need to track where they're going
        counting = False
        tmp = line.split()
        where = tmp[2]
        if where == "..": pass
        else:
            all_dirs.append(where)
            totals[where] = [] 
    if counting:  # We're inside an ls, count up what's in there
        if has_size(line):
            tmp = line.split()
            totals[where].append(int(tmp[0]))  # adding the value the to the list belonging to that dir
        elif is_a_dir(line):
            tmp = line.split()
            totals[where].append(tmp[1])  # add the name of the subdir to the parent dir's list
        elif is_command(line):  # command means we're at the end of the directory and it's time to stop counting!
            counting = False
        else:
            print('panic')
    else:
        if is_calling_ls(line):
            counting = True

len(all_dirs) == len(set(all_dirs)) # not unique of course
        

False

In [ ]:
totals.keys()

In [ ]:
totals.pop('..')
totals.keys()

In [ ]:
dir_values = {key:0 for (key,value) in totals.items()} # can keep track of storage used in each dir in this perhaps

In [ ]:
all_the_keys = set(totals.keys()) # TRACK WHICH KEYS WE'VE LOOKED AT
len(all_the_keys)

In [ ]:
while len(all_the_keys) > 0:
    keys_to_remove = []
    for k in all_the_keys:
        # LOOK IN A DIR, TRY TO ADD UP, IF WE CANT (SUBDIR'S VALUE UNKNOWN), MOVE ON
        contents = totals[k] # list of all inside current dir
        running_total = 0
        for c in contents:
            did_it = True
            if type(c) == int:
                running_total += c
            else:
                val = dir_values[c]  # found a subdir, do we know its value?
                if val == 0:
                    did_it = False
                else:
                    running_total += val
        # So if we managed to figure out the size then we add it to the values dir
        if did_it:
            dir_values[k] = running_total
            running_total = 0
            keys_to_remove.append(k)
    for remove_this_key in keys_to_remove:
        all_the_keys.remove(remove_this_key)
    # DID WE GET THROUGH THEM?    

In [ ]:
final_totals = [val for (key,val) in dir_values.items() if val <= 100000]
sum(final_totals)
